<a href="https://colab.research.google.com/github/hossein-khalilian/Machine-Vision/blob/main/Image_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image segmentation

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive/')
os.chdir("/content/drive/My Drive/app/Image_Segmentation")

In [ ]:
# !wget https://s3.eu-central-1.amazonaws.com/avg-kitti/data_semantics.zip
# !unzip data_semantics.zip

In [ ]:
%tensorflow_version 2.x
import glob
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
from IPython.display import clear_output
from sklearn.model_selection import train_test_split
from __future__ import absolute_import, division, print_function, unicode_literals

tfds.disable_progress_bar()

%tensorflow_version 2.x
import os
import cv2
import glob
import scipy.io
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from google.colab.patches import cv2_imshow
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix
from keras.callbacks import TensorBoard

In [ ]:
sh = (256, 128)
x_train = [cv2.resize(cv2.imread(file), dsize=sh) for file in sorted(glob.glob("dataset/training/image_2/*.png"))]
x_train = np.array(x_train)/255.0
print(x_train.shape)

y_train = [cv2.resize(cv2.imread(file), dsize=sh) for file in sorted(glob.glob("dataset/training/semantic/*.png"))]
y_train = np.array(y_train)
y_train = y_train[:,:,:,0:1]
y_train = y_train.astype('uint8')
print(y_train.shape)

x_test  = [cv2.resize(cv2.imread(file), dsize=sh) for file in sorted(glob.glob("dataset/testing/image_2/*.png"))]
x_test = np.array(x_test)/255.0
print(x_test.shape)

index = np.random.permutation(200)
x_train = x_train[index]
y_train = y_train[index]

In [ ]:
#Defining the model
sh = x_train.shape
output_channels = 34

encoder_input = keras.Input(shape=(sh[1], sh[2], 3), name='original_img')

# x = layers.Conv2D(16, 3, padding='same')(encoder_input)
# x = layers.LeakyReLU()(x)
# x = layers.MaxPooling2D()(x)
# x = layers.Dropout(0.2)(x)

x = layers.Conv2D(32, 3, padding='same')(encoder_input)
x = layers.LeakyReLU()(x)
x = layers.MaxPooling2D()(x)
# x = layers.Dropout(0.2)(x)
x = layers.GaussianDropout(0.1)(x)

x = layers.Conv2D(16, 3, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.MaxPooling2D(4)(x)
# x = layers.Dropout(0.2)(x)
x = layers.GaussianDropout(0.1)(x)
sh1 = x.shape

encoder_output = layers.Flatten()(x)
encoder_output = layers.Dense(256)(encoder_output)
encoder_output = layers.LeakyReLU()(encoder_output)


encoder = keras.Model(encoder_input, encoder_output, name='encoder')
encoder.summary()

decoder_input = keras.Input(shape=256)
x = layers.Dense(sh1[1]*sh1[2]*16)(decoder_input)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)
x = layers.Reshape((sh1[1], sh1[2], 16))(x)

x = layers.Conv2DTranspose(16, 3, strides=4, padding='same', use_bias=False)(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2DTranspose(32, 3, strides=2, padding='same', use_bias=False)(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)

# x = layers.Conv2DTranspose(16, 3, strides=2, padding='same', use_bias=False)(x)
# x = layers.BatchNormalization()(x)
# x = layers.LeakyReLU()(x)

decoder_output = layers.Conv2DTranspose(output_channels, 3, strides=1, padding='same', use_bias=False, activation='sigmoid')(x)


decoder = keras.Model(decoder_input, decoder_output, name='decoder')
decoder.summary()

autoencoder_input = keras.Input(shape=(sh[1], sh[2], 3), name='img')
encoded_img = encoder(autoencoder_input)
decoded_img = decoder(encoded_img)
autoencoder = keras.Model(autoencoder_input, decoded_img, name='autoencoder')
autoencoder.summary()

keras.utils.plot_model(autoencoder, 'my_first_model_with_shape_info.png', show_shapes=True)

In [ ]:
def display(display_list):
    plt.figure(figsize=(15, 15))
    for i in range(len(display_list)): 
        plt.subplot(1, len(display_list), i+1)
        plt.imshow(display_list[i])
    plt.show()
        

def show_predictions():
    y_pred = np.argmax(autoencoder(x_train[0:1]), axis=-1)
    display([x_train[0], y_train[0,:,:,0], y_pred[0]])


class DisplayCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    if epoch % 10 == 0:
        clear_output(wait=True)
        show_predictions()
        print ('\nSample Prediction after epoch {}\n'.format(epoch+1))
    if (epoch + 1) % 50 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)

In [ ]:
autoencoder.compile(optimizer=keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['acc'])

In [ ]:
checkpoint_dir = './training_checkpoints_image_segmentation_with_gausian_dropout50'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")

checkpoint = tf.train.Checkpoint(model=autoencoder)

# checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

### Training

In [ ]:
skip_training = False
batch_size = 5
epochs = 300

logdir = './log'+'/dropout'
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

if not skip_training:
    model_history = autoencoder.fit(x_train, y_train, verbose=False,
            batch_size=batch_size,
            epochs=epochs,
            callbacks=[DisplayCallback(), tensorboard_callback],
            validation_split=0.2,
            )
#     autoencoder.save('autoencoder_image_segmentation.h5') 
if skip_training:
    checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [ ]:
print(model_history.history['val_acc'][-1])
print(model_history.history['acc'][-1])

In [ ]:
# autoencoder = tf.keras.models.load_model('autoencoder_image_segmentation_final.h5')
# checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))
x_out = autoencoder(x_train[100:110])
x_out = np.argmax(x_out, axis=3)
for x in x_out:
    plt.imshow(x)
    plt.show()
x_out = autoencoder(x_test[0:5])
x_out = np.argmax(x_out, axis=3)
for x in x_out:
    plt.imshow(x)
    plt.show()

In [ ]:
y_pred = autoencoder(x_train)
y_pred = np.argmax(y_pred, axis=3)
y1 = y_train[:5].reshape(-1,1)
y2 = y_pred[:5].reshape(-1, 1)
cnf_matrix = confusion_matrix(y1, y2)
np.set_printoptions(linewidth=np.inf)
print(cnf_matrix.shape)
# print(cnf_matrix)

In [ ]:
# ! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
# ! unzip ngrok-stable-linux-amd64.zip

In [ ]:
logdir="log"
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(logdir)
)
get_ipython().system_raw('./ngrok http 6006 &')

In [ ]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [ ]:
# # %load_ext tensorboard 
# %tensorboard --logdir log

In [ ]:
# from tensorboard import notebook
# notebook.list() 
# notebook.display(port=6006, height=2000) 